In [5]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Path to your .db file
db_path = 'C:\\Users\\Joey\\PycharmProjects\\Machine Learning\\Data\\grading_equation_reference_data_all.db'

# Establishing a connection to the database
conn = sqlite3.connect(db_path)

# Get the list of all columns in the 'variables' table
column_query = "PRAGMA table_info(variables)"
columns_info = pd.read_sql_query(column_query, conn)

# Assuming the first column is an ID or similar and you want columns 4 through 22 (adjust as needed)
# Column indexes in the DataFrame start from 0, so for columns 4 through 22 you actually want indexes 3 through 21
desired_columns = columns_info.loc[3:21, 'name'].tolist()

# Adding 'MPH' to the beginning of the list if it's not already included and you need it for your analysis
if 'MPH' not in desired_columns:
    desired_columns.insert(0, 'MPH')

# Constructing the SQL query with the desired columns
query = f"SELECT {', '.join(desired_columns)} FROM variables"

# Reading the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Now, df contains your data, and you can proceed with data preparation and model training

# Assuming you have already loaded your dataframe 'df' from the database

# Handling NaN values in MPH - choose one of the methods mentioned above
df = df.dropna(subset=['MPH'])  # Method 1: Dropping rows with NaN in MPH
# OR
df['MPH'] = df['MPH'].fillna(df['MPH'].median())  # Method 2: Imputation

# Assuming df is loaded and contains both numeric and non-numeric data

# Splitting the dataset into training and testing sets first
X = df.drop('MPH', axis=1)  # Predictor variables, before removing or encoding non-numeric columns
y = df['MPH']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identifying non-numeric columns in the training set
non_numeric_columns_train = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

# Option 1: Drop non-numeric columns from both training and testing sets
X_train = X_train.drop(columns=non_numeric_columns_train)
X_test = X_test.drop(columns=non_numeric_columns_train)

# Option 2: Convert categorical variables to numeric
# Note: For one-hot encoding, you need to ensure that the same dummy variables are present in both training and testing sets.
# This might require adjusting after encoding if the train and test sets have different categories.
# This is a more complex scenario that often requires careful alignment of columns after encoding.

# Proceed with model training using the adjusted X_train and X_test
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions and evaluations
y_pred = model.predict(X_test)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Adjusting the line for creating the coefficients DataFrame to use X_train.columns
coefficients = pd.DataFrame(model.coef_, X_train.columns, columns=['Coefficient'])
print(coefficients)



RMSE: 5.232328075332154e-14
R^2: 1.0
                       Coefficient
Score                 1.000000e+00
Linear_Pelvis_Speed  -5.000000e-02
HSS_Footplant         3.151027e-15
Pelvis_Ang_Footplant  4.500000e-01
Trunk_Ang_Footplant  -6.000000e-01
Pelvic_Obl           -5.000000e-02
Front_Leg_Brace      -1.500000e-01
Front_Leg_Var_Val    -1.000000e-01
Lead_Leg_Midpoint    -5.000000e+00
Lead_Leg_GRF_y       -4.344394e-15
Lead_Leg_GRF_z       -6.898697e-15
Lead_Leg_GRF_x        1.661614e-14
Horizontal_Abduction -5.500000e-01
Shld_ER_Footplant    -6.669062e-17
Shld_ER_Max          -1.000000e-01
Lateral_Trunk_Tilt    1.434560e-15
Pelvis_Ang_Velo      -5.000000e-02
Torso_Ang_Velo       -1.200000e-02


In [6]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Assuming you have already loaded your dataframe 'df' from the database

# Handling NaN values in MPH - choose one of the methods mentioned above
df = df.dropna(subset=['MPH'])  # Method 1: Dropping rows with NaN in MPH
# OR
df['MPH'] = df['MPH'].fillna(df['MPH'].median())  # Method 2: Imputation

# Data Preparation
X = df.drop('MPH', axis=1)  # Predictor variables
y = df['MPH']  # Target variable

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Feature Importance for Linear Regression
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)


ValueError: could not convert string to float: 'Fastball RH 6.c3d'

In [23]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Path to your .db file
db_path = 'C:\\Users\\Joey\\PycharmProjects\\Machine Learning\\Data\\grading_equation_reference_data_PtoP.db'

# Establishing a connection to the database
conn = sqlite3.connect(db_path)
# Get the list of all columns in the 'variables' table
column_query = "PRAGMA table_info(variables)"
columns_info = pd.read_sql_query(column_query, conn)

# Assuming the first column is an ID or similar, and 'MPH' is the target variable
# You want to include 'MPH' explicitly and then add columns 4 through 22 by their names
# Note: Adjust the indices 3:22 based on the actual positions of your desired columns
desired_columns = ['MPH'] + columns_info.loc[5:21, 'name'].tolist()

# Constructing the SQL query with the desired columns
query = f"SELECT {', '.join(desired_columns)} FROM variables"

# Reading the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Remove rows where MPH is NaN
df = df.dropna(subset=['MPH'])

# Handling non-numeric data: Convert categorical variables using one-hot encoding
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Splitting the dataset
X = df.drop('MPH', axis=1)
y = df['MPH'].valuesgit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"R^2: {r2}")

# Feature Importance
feature_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

RMSE: 2.497522772668947
R^2: -0.012600649350647197
                      importance
HSS_Footplant           0.122413
Lateral_Trunk_Tilt      0.115831
Lead_Leg_Midpoint       0.114041
Pelvis_Ang_Velo         0.102126
Linear_Pelvis_Speed     0.097265
Pelvis_Ang_Footplant    0.095542
Pelvic_Obl              0.082396
Shld_ER_Footplant       0.080754
Front_Leg_Brace         0.050059
Trunk_Ang_Footplant     0.046564
Horizontal_Abduction    0.033961
Shld_ER_Max             0.024659
Lead_Leg_GRF_y          0.011390
Lead_Leg_GRF_x          0.010089
Lead_Leg_GRF_z          0.008983
Front_Leg_Var_Val       0.003307
Torso_Ang_Velo          0.000619
